In [11]:
import random
from copy import deepcopy

########### Low level functions ##########
## Get random name from list
def get_random_name(list):
    index = random.randrange(0, len(list))
    return list[index]

## Remove name from list
def remove_name_from_list(list, remove_name):
    shortened_list = deepcopy(list)
    if remove_name in shortened_list:
        shortened_list.remove(remove_name)
    return shortened_list

## Check if assignments work
def check_assignments(assignment_dict1, assignment_dict2):
    for key in assignment_dict1:
        if (assignment_dict1[key] == assignment_dict2[key]):
            return False
    return True    
    
########### High level functions ##########
def get_name_assignment(full_name_list, assignment_dict):
    recipient_list = deepcopy(full_name_list)
    
    ## Assign recipients
    for i in range(0, len(full_name_list)):
        ## Get buyer
        buyer = full_name_list[i]

        ## If recipient list only has buyer left, then failed
        shortened_list = remove_name_from_list(recipient_list, buyer)
        if (len(shortened_list) == 0):
            assignment_dict = {}
            return False
        
        ## Get random recipient, excluding buyer
        recipient = get_random_name(shortened_list)

        ## Remove recipient from recipient list
        recipient_list.remove(recipient)

        ## Add buyer-> recipient pair
        assignment_dict[buyer] = recipient
    return True

In [14]:
import smtplib

## Send email
def sendEmail(TO, SUBJECT, TEXT, USER= "learningpython640@gmail.com", PASSWORD= "ilearnpython"):
    ## Setup email content
    to = [TO]
    email_text = """\
    From: %s  
    To: %s  
    Subject: %s

    %s
    """ % (USER, ", ".join(to), SUBJECT, TEXT)
    
    ## Setup Server connection
    try:
        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.ehlo()
        server.login(USER, PASSWORD)
        server.sendmail(USER, to, email_text)
        server.close()

        print("Email sent")
    except:
        print("Something went wrong")
        
sendEmail("tylergwlum@gmail.com", "OMG Super Important Message", "Miss you dude")

Email sent


In [12]:
########### Program ##########
NAME_LIST = ["Tyler", "Nancy", "Jenny", "Sandy", "Kent", "Peter"]  # do not change
assignment_dict1 = {}
success = False
while(not success):
    success = get_name_assignment(NAME_LIST, assignment_dict1)

assignment_dict2 = {}
while(True):
    success = False
    while(not success):
        success = get_name_assignment(NAME_LIST, assignment_dict2)
    if (check_assignments(assignment_dict1, assignment_dict2)):
        break

print(assignment_dict1)
print(assignment_dict2)

final_assignment_dict = {}
for key in assignment_dict1:
    list = [assignment_dict1[key], assignment_dict2[key]]
    final_assignment_dict[key] = list
print(final_assignment_dict)

{'Tyler': 'Kent', 'Nancy': 'Tyler', 'Jenny': 'Sandy', 'Sandy': 'Peter', 'Kent': 'Jenny', 'Peter': 'Nancy'}
{'Tyler': 'Sandy', 'Nancy': 'Peter', 'Jenny': 'Nancy', 'Sandy': 'Kent', 'Kent': 'Tyler', 'Peter': 'Jenny'}
{'Tyler': ['Kent', 'Sandy'], 'Nancy': ['Tyler', 'Peter'], 'Jenny': ['Sandy', 'Nancy'], 'Sandy': ['Peter', 'Kent'], 'Kent': ['Jenny', 'Tyler'], 'Peter': ['Nancy', 'Jenny']}
